In [1]:
# Cell 1: Imports and Setup
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pickle
import json
from datetime import datetime
from pathlib import Path

class MetronomeTime:
    def __init__(self, tick_rate=0.01):
        self.current_tick = 0
        self.tick_rate = tick_rate
        self.lobe_states = {}
        
    def register_lobe(self, lobe_id):
        self.lobe_states[lobe_id] = False
        
    def tick(self):
        if all(self.lobe_states.values()):
            self.current_tick += self.tick_rate
            self.lobe_states = {k: False for k in self.lobe_states}
            return True
        return False
        
    def save_state(self, path):
        state = {
            'current_tick': self.current_tick,
            'tick_rate': self.tick_rate
        }
        with open(path, 'wb') as f:
            pickle.dump(state, f)

In [2]:
# Cell 2: Harmonic Embeddings
class HarmonicEmbedding(nn.Module):
    def __init__(self, embedding_dim, num_harmonics):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.num_harmonics = num_harmonics
        self.frequencies = nn.Parameter(torch.randn(num_harmonics))
        self.phases = nn.Parameter(torch.randn(num_harmonics))
        
    def forward(self, t):
        # Generate harmonic basis
        t = t.unsqueeze(-1)
        harmonics = torch.sin(2 * np.pi * self.frequencies * t + self.phases)
        return harmonics
    
    def save_embeddings(self, path):
        state = {
            'frequencies': self.frequencies.detach().cpu().numpy(),
            'phases': self.phases.detach().cpu().numpy()
        }
        torch.save(state, path)
        
    def load_embeddings(self, path):
        state = torch.load(path)
        self.frequencies.data = torch.tensor(state['frequencies'])
        self.phases.data = torch.tensor(state['phases'])

In [3]:
# Cell 3: LTC Neuron with Persistence
from typing import Tuple, Optional
import torch
import torch.nn as nn
class LiquidTimeConstant(nn.Module):
    def __init__(self, input_size: int, hidden_size: int):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        # Backbone network
        self.backbone = nn.Sequential(
            nn.Linear(input_size + hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size)
        )
        
        # Networks for liquid time-constant mechanism
        self.time_net = nn.Linear(hidden_size, hidden_size)
        self.state_net_g = nn.Linear(hidden_size, hidden_size)
        self.state_net_h = nn.Linear(hidden_size, hidden_size)
        
        # Learnable parameters
        self.tau = nn.Parameter(torch.ones(hidden_size))
        self.A = nn.Parameter(torch.randn(hidden_size))

    def forward(self, x: torch.Tensor, h: torch.Tensor, t: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        # Create new tensor instead of modifying in place
        h_new = h.clone()
        
        # Combine inputs using non-inplace operations
        combined = torch.cat([x, h_new], dim=-1)
        features = self.backbone(combined)
        
        # Compute time-constant factor
        f_t = torch.sigmoid(self.time_net(features))
        
        # State transformations
        g_x = self.state_net_g(features)
        h_x = self.state_net_h(features)
        
        # Time-dependent gating using non-inplace operations
        t_view = t.view(-1, 1)
        gate = torch.sigmoid(torch.mul(-f_t, t_view))
        
        # Create new tensors for intermediate computations
        gate_g = torch.mul(gate, g_x)
        inverse_gate = torch.sub(torch.ones_like(gate), gate)
        gate_h = torch.mul(inverse_gate, h_x)
        
        # Combine outputs into new tensor
        output = torch.add(gate_g, gate_h)
        
        return output, output

In [4]:
class TransformerLNN(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, num_heads: int = 4, dropout: float = 0.1):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        
        # Layers
        self.input_proj = nn.Linear(input_size, hidden_size)
        self.attention = nn.MultiheadAttention(hidden_size, num_heads, dropout=dropout)
        self.ltc = LiquidTimeConstant(hidden_size, hidden_size)
        self.output_proj = nn.Linear(hidden_size, input_size)
        
        # Layer normalization
        self.norm1 = nn.LayerNorm(hidden_size)
        self.norm2 = nn.LayerNorm(hidden_size)

    def forward(self, x: torch.Tensor, times: Optional[torch.Tensor] = None, 
                mask: Optional[torch.Tensor] = None) -> torch.Tensor:
        batch_size, seq_len, _ = x.shape
        
        if times is None:
            times = torch.arange(seq_len, dtype=torch.float32, device=x.device)
            times = times.unsqueeze(0).expand(batch_size, -1)
        
        # Project input
        h = self.input_proj(x)
        
        # Self-attention
        h_att = h.transpose(0, 1)
        h_att, _ = self.attention(h_att, h_att, h_att, attn_mask=mask)
        h_att = h_att.transpose(0, 1)
        h_att = self.norm1(h + h_att)
        
        # LTC processing
        ltc_state = torch.zeros(batch_size, self.hidden_size, device=x.device)
        outputs = []
        
        for t in range(seq_len):
            ltc_in = h_att[:, t]
            ltc_out, ltc_state = self.ltc(ltc_in, ltc_state, times[:, t])
            outputs.append(ltc_out)
        
        outputs = torch.stack(outputs, dim=1)
        outputs = self.norm2(outputs + h_att)
        
        return self.output_proj(outputs)

    def get_attention_weights(self, x: torch.Tensor) -> torch.Tensor:
        h = self.input_proj(x)
        h = h.transpose(0, 1)
        _, attn_weights = self.attention(h, h, h, need_weights=True)
        return attn_weights

In [5]:
class PersistentLTCNeuron(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.ltc = LiquidTimeConstant(input_size, hidden_size)
        self.state_history = []
        self.lobe_id = None
        
    def forward(self, x: torch.Tensor, h: torch.Tensor, t: torch.Tensor, metronome: MetronomeTime):
        if self.lobe_id is not None:
            metronome.lobe_states[self.lobe_id] = True
        
        # Forward pass through LTC with proper arguments
        output, new_state = self.ltc(x, h, t)
        
        # Record state history
        self.state_history.append({
            'time': metronome.current_tick,
            'state': new_state.detach().cpu().numpy()
        })
        
        return output, new_state
        
    def save_state(self, path):
        state = {
            'model_state': self.ltc.state_dict(),
            'history': self.state_history
        }
        torch.save(state, path)
        
    def load_state(self, path):
        state = torch.load(path)
        self.ltc.load_state_dict(state['model_state'])
        self.state_history = state['history']

In [ ]:
class HybridEmbeddingLayer(nn.Module):
    def __init__(self, vocab_size: int, embedding_dim: int, num_harmonics: int):
        super().__init__()
        self.transformer_embedding = nn.Embedding(vocab_size, embedding_dim)
        self.harmonic_embedding = HarmonicEmbedding(embedding_dim, num_harmonics)
        self.phase = "transformer"  # Controls training phase
        self.embedding_memory = []
        
    def forward(self, x: torch.Tensor, t: torch.Tensor):
        if self.phase == "learning":
            # Collect transformer embeddings during initial phase
            trans_emb = self.transformer_embedding(x)
            self.embedding_memory.append(trans_emb.detach())
            return trans_emb
            
        elif self.phase == "transition":
            # Use both embeddings and train harmonic LTC
            trans_emb = self.transformer_embedding(x)
            harm_emb = self.harmonic_embedding(t)
            return trans_emb + 0.5 * harm_emb
            
        else:  # harmonic phase
            return self.harmonic_embedding(t)

In [ ]:
class LTCEmbeddingChain(nn.Module):
    def __init__(self, hidden_size: int, chain_length: int):
        super().__init__()
        self.ltc_chain = nn.ModuleList([
            LiquidTimeConstant(hidden_size, hidden_size) 
            for _ in range(chain_length)
        ])
        
    def forward(self, x: torch.Tensor, t: torch.Tensor):
        h = torch.zeros_like(x)
        for ltc in self.ltc_chain:
            x, h = ltc(x, h, t)
        return x

In [6]:
def visualize_training(t, y_true, y_pred, losses, times, harmonic_embedding):
    plt.figure(figsize=(20, 10))
    
    # Plot 1: Training Loss
    plt.subplot(231)
    plt.plot(times, losses)
    plt.title('Training Loss')
    plt.xlabel('Metronome Time')
    plt.ylabel('Loss')
    plt.grid(True)
    
    # Plot 2: Predictions
    plt.subplot(232)
    plt.plot(t.detach().cpu(), y_true.detach().cpu(), label='True')
    plt.plot(t.detach().cpu(), y_pred.detach().cpu(), label='Predicted')
    plt.title('Predictions vs Ground Truth')
    plt.legend()
    plt.grid(True)
    
    # Plot 3: Harmonic Components
    plt.subplot(233)
    with torch.no_grad():
        harmonics = harmonic_embedding(t)
        harmonics = harmonics.detach().cpu()
    plt.plot(t.detach().cpu(), harmonics)
    plt.title('Harmonic Components')
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()


In [7]:
def setup_training(input_size=10, hidden_size=64, num_heads=4, 
                  batch_size=32, learning_rate=1e-3):
    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Initialize model
    model = TransformerLNN(
        input_size=input_size,
        hidden_size=hidden_size,
        num_heads=num_heads
    ).to(device)
    
    # Loss and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    return model, criterion, optimizer, device

In [8]:
def generate_synthetic_data(num_samples: int = 1000,
                          seq_length: int = 50,
                          input_dim: int = 10,
                          device: str = 'cuda' if torch.cuda.is_available() else 'cpu'
                          ) -> Tuple[torch.Tensor, torch.Tensor]:
    t = torch.linspace(0, 10, seq_length, device=device)
    t = t.view(1, -1, 1).repeat(num_samples, 1, input_dim)
    
    patterns = []
    for i in range(input_dim):
        freq1 = (i + 1) * 0.5
        freq2 = (i + 1) * 0.25
        pattern = torch.sin(2 * np.pi * freq1 * t[..., i]) + \
                 0.5 * torch.sin(2 * np.pi * freq2 * t[..., i])
        patterns.append(pattern)
    
    x = torch.stack(patterns, dim=-1)
    y = torch.roll(x, shifts=-1, dims=1) * 1.5 + 0.5
    
    return x, y

In [10]:
def train_ltc_with_harmonics(ltc_neuron, harmonic_embedding, metronome, num_epochs=100, plot_interval=10):
    optimizer = optim.Adam(list(ltc_neuron.parameters()) + 
                          list(harmonic_embedding.parameters()))
    
    data_logger = DataLogger()
    losses = []
    times = []
    
    # Initialize hidden state
    batch_size = 32
    h = torch.zeros(batch_size, ltc_neuron.ltc.hidden_size)
    
    for epoch in range(num_epochs):
        # Generate time steps
        t = torch.linspace(0, 10, batch_size)
        
        # Forward pass through harmonic embedding
        x = harmonic_embedding(t)
        
        # Create new hidden state for each forward pass
        h_current = h.clone().detach()
        
        # Forward pass through LTC
        y_pred, new_h = ltc_neuron(x, h_current, t, metronome)
        y_true = torch.sin(t).view(-1, 1)
        
        # Compute loss
        loss = nn.MSELoss()(y_pred, y_true)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Update hidden state after backward pass
        h = new_h.detach()
        
        # Log metrics
        data_logger.update_metrics(
            epoch=epoch,
            batch=0,
            loss=loss.item(),
            y_pred=y_pred,
            y_true=y_true,
            ltc_state=h,
            attn_weights=torch.zeros(1)  # Placeholder for attention weights
        )
        
        losses.append(loss.item())
        times.append(metronome.current_tick)
        
        if epoch % plot_interval == 0:
            data_logger.display_recent_metrics()
            metronome.tick()
    
    return losses, times

class DataLogger:
    def __init__(self):
        self.metrics = {
            'epoch': [],
            'batch': [],
            'loss': [],
            'accuracy': [],
            'ltc_state_norm': [],
            'timestamp': []
        }
    
    def update_metrics(self, epoch, batch, loss, y_pred, y_true, ltc_state, attn_weights):
        mse = ((y_pred - y_true) ** 2).mean().item()
        accuracy = 100 * (1 - min(mse, 1))
        
        self.metrics['epoch'].append(epoch)
        self.metrics['batch'].append(batch)
        self.metrics['loss'].append(f"{loss:.6f}")
        self.metrics['accuracy'].append(f"{accuracy:.2f}")
        self.metrics['ltc_state_norm'].append(f"{torch.norm(ltc_state).item():.4f}")
        self.metrics['timestamp'].append(datetime.now().strftime("%H:%M:%S"))
    
    def display_recent_metrics(self, n_rows: int = 10):
        headers = ['Epoch', 'Batch', 'Loss', 'Acc(%)', 'LTC Norm', 'Time']
        
        data = [
            [self.metrics[k][-i] for k in self.metrics.keys()]
            for i in range(min(n_rows, len(self.metrics['epoch'])), 0, -1)
        ]
        
        print("\nRecent Training Metrics:")
        print("-" * 80)
        print(f"{headers[0]:<6} {headers[1]:<6} {headers[2]:<10} {headers[3]:<8} "
              f"{headers[4]:<10} {headers[5]:<8}")
        print("-" * 80)
        
        for row in data:
            print(f"{row[0]:<6} {row[1]:<6} {row[2]:<10} {row[3]:<8} "
                  f"{row[4]:<10} {row[5]:<8}")

In [11]:
# Cell 5: Example Usage
# Initialize components
metronome = MetronomeTime(tick_rate=0.01)
harmonic_embed = HarmonicEmbedding(embedding_dim=32, num_harmonics=8)
ltc_neuron = PersistentLTCNeuron(input_size=8, hidden_size=32)

# Register LTC neuron with metronome
ltc_neuron.lobe_id = 'lobe_1'
metronome.register_lobe('lobe_1')
torch.autograd.set_detect_anomaly(True)
# Train and visualize
losses, times = train_ltc_with_harmonics(
    ltc_neuron, 
    harmonic_embed,
    metronome,
    num_epochs=1000
)

# Save trained components
save_dir = Path('trained_components')
save_dir.mkdir(exist_ok=True)

metronome.save_state(save_dir / 'metronome.pkl')
harmonic_embed.save_embeddings(save_dir / 'harmonics.pt')
ltc_neuron.save_state(save_dir / 'ltc.pt')

/Volumes/stuff/solace/Kognitive/python_proofs/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)



Recent Training Metrics:
--------------------------------------------------------------------------------
Epoch  Batch  Loss       Acc(%)   LTC Norm   Time    
--------------------------------------------------------------------------------
0      0      0.476642   52.34    4.1856     20:26:16

Recent Training Metrics:
--------------------------------------------------------------------------------
Epoch  Batch  Loss       Acc(%)   LTC Norm   Time    
--------------------------------------------------------------------------------
1      0      0.473311   52.67    4.1262     20:26:16
2      0      0.469977   53.00    4.0472     20:26:16
3      0      0.466469   53.35    3.9614     20:26:16
4      0      0.462935   53.71    3.8926     20:26:17
5      0      0.459334   54.07    3.8371     20:26:17
6      0      0.455643   54.44    3.7953     20:26:17
7      0      0.451836   54.82    3.7674     20:26:17
8      0      0.447881   55.21    3.7536     20:26:17
9      0      0.443747   55.63